In [1]:
import logging
import re

import numpy as np
import pandas as pd

import cipy
import textacy

In [2]:
logger = logging.getLogger('cipy')
logger.setLevel(logging.DEBUG)

In [3]:
conn_creds = cipy.db.get_conn_creds('DATABASE_URL')
citations_db = cipy.db.PostgresDB(conn_creds, ddl='citations')

In [4]:
citations_db.ddl

DDL(table_name='citations', path='/Users/burtondewilde/Desktop/datakind/ci/conservation-intl/cipy/ddls/citations.yaml')

In [5]:
citations_db.ddl.create_index_statements()

['CREATE INDEX IF NOT EXISTS citations_project_id_idx ON citations USING btree (project_id)']

In [8]:
citations_db.ddl['schema']['indexes']

[{'column_name': 'project_id',
  'index_name': 'citations_project_id_idx',
  'unique': False}]

In [6]:
project_id = cipy.hack.get_project_id()
project_id

0

In [15]:
query = """
SELECT
    citation_id, title, abstract, keywords, publication_year, authors
FROM citations
WHERE
    project_id = %(project_id)s
    AND (citation_id NOT IN (SELECT citation_id FROM duplicates)
         OR citation_id IN (SELECT canonical_citation_id FROM duplicates))
"""

query = """
SELECT
    citation_id,
    (CASE WHEN abstract IS NOT NULL THEN title || '\n\n' || abstract ELSE '' END) AS text,
    keywords, publication_year, authors
FROM citations
WHERE
    project_id = %(project_id)s
    AND (citation_id NOT IN (SELECT citation_id FROM duplicates)
         OR citation_id IN (SELECT canonical_citation_id FROM duplicates))
"""

query = """
SELECT
    citation_id,
    TRIM('\n' FROM concat_ws('\n\n', COALESCE(title, ''),
                   COALESCE(abstract, ''),
                   COALESCE(array_to_string(keywords, ', '), ''))) AS text,
    publication_year,
    authors
FROM citations
WHERE
    project_id = %(project_id)s
    AND (citation_id NOT IN (SELECT citation_id FROM duplicates)
         OR citation_id IN (SELECT canonical_citation_id FROM duplicates))
LIMIT 10
"""

df = pd.DataFrame(citations_db.run_query(query, {'project_id': project_id}))

In [16]:
print(df.shape)
df.head()

(10, 4)


,authors,citation_id,publication_year,text
0,"[113 Congress, Rep. McDermott]",1,NaN,"Managed Carbon Price Act of 2014, H.R. 4754"
1,"[113 Congress, Rep. McDermott]",2,NaN,"Managed Carbon Price Act of 2014, H.R. 4754"
2,"[450/3-90-023, E P A]",3,1991.0,Evaluating Exposures to Toxic Air Pollutants: ...
3,"[Aagaard, T S]",4,2006.0,A fresh look at the responsible relation doctr...
4,"[Aagesen, D]",5,2000.0,Crisis and conservation at the end of the worl...


In [27]:
print(df.ix[4, 'text'])

Crisis and conservation at the end of the world: Sheep ranching in Argentine Patagonia

Patagonia was one of the last regions in the Americas to be settled by Europeans. It was not until the mid-1880s that the Argentine government secured effective control over the region, after which settlement, and economic development were based on sheep ranching. Virtually free of domesticated animals in 1885, by 1910 Patagonian rangelands supported some 12 million sheep. This growth was sustained until 1952, when the sheep population of the region peaked at approximately 22 million, but the number of sheep in Patagonia has since declined to about 13 million. Numerous factors have been implicated in the collapse of sheep ranching, one of great significance being the very poor state of Patagonian rangelands. Soil erosion is widespread, and the flora has been so heavily modified that it is extremely difficult, if not impossible, to determine what the composition and characteristics of Patagonia's pre

In [10]:
%%time
records = pgdb.run_query(query, {'project_id': project_id})
text_stream, metadata_stream = textacy.fileio.split_content_and_metadata(records, 'text', itemwise=False)

# corpus = textacy.TextCorpus.from_texts('en', text_stream, metadata=metadata_stream)

spacy_pipeline = textacy.data.load_spacy('en')#, parser=False)
corpus = textacy.TextCorpus(spacy_pipeline)
for text, metadata in zip(text_stream, metadata_stream):
    corpus.add_text(text, lang='en', metadata=metadata)
corpus

CPU times: user 8min 18s, sys: 2.96 s, total: 8min 21s
Wall time: 8min 21s


In [7]:
%%time
records = pgdb.run_query(query, {'project_id': project_id})
text_stream, metadata_stream = textacy.fileio.split_content_and_metadata(records, 'text', itemwise=False)

# corpus = textacy.TextCorpus.from_texts('en', text_stream, metadata=metadata_stream)

spacy_pipeline = textacy.data.load_spacy('en', parser=False)
corpus = textacy.TextCorpus(spacy_pipeline)
for text, metadata in zip(text_stream, metadata_stream):
    corpus.add_text(text, lang='en', metadata=metadata)
corpus

CPU times: user 1min 58s, sys: 1.83 s, total: 2min
Wall time: 2min


In [8]:
corpus

TextCorpus(28343 docs; 7806511 tokens)

In [26]:
keyterms = {'intervention': {'conservation', 'conserve', 'conservancy',
                             'protect*', 'management', 'awareness', 'law*',
                             'policy*', 'reserve*', 'govern*', 'capacity-build*',
                             'train*', 'regulation', 'payment for ecosystem services', 'PES',
                             'ecotourism', 'sustainable use'},
            'outcome': {'wellbeing', 'well-being', 'well being', 'ecosystem service*',
                        'nutrition', 'skill*', 'empower*', 'clean water', 'livelihood*',
                        'food security', 'resilience*', 'vulnerability', 'social capital',
                        'attitude*', 'perception*', '(human)? health*', 'human capital',
                        '(traditional)? knowledge'},
            'intervention_qualifiers': {'marine', 'freshwater', 'coastal', 'forest*',
                                        'ecosystem*', 'species', 'habitat*', 'biodiversity',
                                        'sustainab*', 'ecolog*', 'integrated', 'landscape',
                                        'seascape', 'coral reef*', 'natural resource*'},
            'outcome_qualifiers': {'human*', 'people', 'person*', 'communit*',
                                   'household*', 'fisher*', 'collaborative'}
            }

keyterms_joined = '|'.join(val.replace('*', '*?')
                           for vals in keyterms.values()
                           for val in vals)
keyterms_re = re.compile(r'(?<=^|\b)(' + keyterms_joined + r')(?=$|\b)', flags=re.IGNORECASE)

In [37]:
match_fractions = []

for doc in corpus:
    if not doc.text:
        match_fractions.append(0.0)
        continue
    match_fractions.append(
        sum(len(match.group()) for match in keyterms_re.finditer(doc.text)) / len(doc.text))

In [46]:
match_fraction_idxs = np.argsort(match_fractions)[::-1]
match_fraction_idxs

array([ 3931, 23503, 14842, ..., 10493, 10513,     0])

In [49]:
for idx in match_fraction_idxs[:5]:
    print('\n')
    print(corpus[idx].text)



Cultural erosion and biodiversity: Canoe-making knowledge in Pohnpei, Micronesia

Erosion of traditional knowledge and practice is a serious and accelerating problem, but quantitative work on traditional knowledge loss and its importance to biodiversity conservation is lacking. We investigated traditional knowledge of canoe making, a skill heavily dependent on plant biodiversity, on Pohnpei, Federated States of Micronesia, through a survey of 180 island residents. Our results showed that there has been an intergenerational erosion of canoe-making skills. Given current trends, the present generation of Pohnpeians may be the last to retain any knowledge of this traditional craft. We also identified several correlates of knowledge loss - including Western educational level and occupation - that highlight potential avenues for skill conservation via governments, traditional leadership, and schools. These institutions could intervene to emphasize traditional knowledge, which would reinfor

In [51]:
for idx in match_fraction_idxs[-8:]:
    print('\n')
    print(corpus[idx].text)






Using land-time-budgets to analyse farming systems and poverty alleviation policies in the Lao PDR

This paper applies the method of 'Land-time-budget analysis' to a rural subsistence community and to the national economy of the Lao PDR. The analysis is conducted to meet two ends: • To identify the community's/the nation's resource use profile in terms of land and time use. The analysis identifies biophysical constraints of socio-economic development and trade-offs in resource use patterns. • To contrast the results of the analysis with national poverty alleviation policies and visualise their effects on local communities. Results show that shifting cultivation, a traditional socio-economic strategy in Laos, is doomed for extinction as a practice for securing subsistence. Little, if any, provisions are made by the planners to persuade shifting cultivators to leave their trade and moving to the lowlands and urban areas. Policies are shown to actually decrease the rate of subsistenc

In [52]:
keyterms = {'intervention': {'conservation', 'conserve', 'conservancy',
                             'protect*', 'management', 'awareness', 'law*',
                             'policy*', 'reserve*', 'govern*', 'capacity-build*',
                             'train*', 'regulation', 'payment for ecosystem services', 'PES',
                             'ecotourism', 'sustainable use'},
            'outcome': {'wellbeing', 'well-being', 'well being', 'ecosystem service*',
                        'nutrition', 'skill*', 'empower*', 'clean water', 'livelihood*',
                        'food security', 'resilience*', 'vulnerability', 'social capital',
                        'attitude*', 'perception*', '(human)? health*', 'human capital',
                        '(traditional)? knowledge'},
            'intervention_qualifiers': {'marine', 'freshwater', 'coastal', 'forest*',
                                        'ecosystem*', 'species', 'habitat*', 'biodiversity',
                                        'sustainab*', 'ecolog*', 'integrated', 'landscape',
                                        'seascape', 'coral reef*', 'natural resource*'},
            'outcome_qualifiers': {'human*', 'people', 'person*', 'communit*',
                                   'household*', 'fisher*', 'collaborative'}
            }

keyterms_joined = '|'.join(val.replace('*', '*?')
                           for vals in keyterms.values()
                           for val in vals)
keyterms_re = re.compile(r'(?<=^|\b)(' + keyterms_joined + r')(?=$|\b)', flags=re.IGNORECASE)

records = pgdb.run_query(query, {'project_id': project_id})

match_fractions = []
for record in records:
    if not record['text']:
        match_fractions.append(0.0)
        continue
    match_fractions.append(
        sum(len(match.group()) for match in keyterms_re.finditer(record['text'])) / len(record['text']))

match_fraction_idxs = np.argsort(match_fractions)[::-1]

In [53]:
match_fraction_idxs

array([ 3931, 23503, 14842, ..., 10493, 10513,     0])